In [42]:
# import libraries 
import numpy as np 
import pandas as pd 
import tensorflow as tf
import os

import matplotlib.pyplot as plt

from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

from skimage import io, color
from skimage.feature import local_binary_pattern
from skimage.feature import canny
from skimage.filters import sobel, sobel_h, sobel_v
from scipy.stats import entropy

from sklearn.svm import SVC


In [3]:
# calculate average size to help inform a size to resize all images to
imagex = 0
imagey = 0
for i in range(1, 5489):    
    image = Image.open(f'Data/train/img_00{i:04d}.jpg')
    imagex += image.size[0]
    imagey += image.size[1]
    
print(imagex/5488) # 50.567237609329446
print(imagey/5488) # 50.05648688046647

# scale to 50 x 50
rescale_size = (50, 50)

50.567237609329446
50.05648688046647


In [4]:
# import paths and labels
train_metadata = pd.read_csv("Data/train/train_metadata.csv", index_col="id")
train_paths = train_metadata["image_path"]
train_class = train_metadata["ClassId"]

# import test set
test_metadata = pd.read_csv("Data/test/test_metadata.csv")
test_paths = test_metadata["image_path"]
test_index = test_metadata["id"]

rescale_size = (50, 50)

# Neural Network

In [ ]:
# import image files as pixel RGB values

train_data = []
train_labels = []

for i in range(len(train_paths)):
    try:
        # open image
        image = Image.open(f"Data/train/{train_paths.iloc[i]}")
        # resize image to (50, 50)
        image = image.resize(rescale_size)
        image = np.array(image)
        train_data.append(image)
        train_labels.append(train_class.iloc[i])
    except:
        print(f"error loading image {train_paths.iloc[i]}")

data = np.array(train_data)
labels = np.array(train_labels)

In [ ]:
# sanity check data

print(f"data has shape: {data.shape}")
print(f"labels have shape: {labels.shape}")

# split into train test sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=69)

print(f"X_train has shape: {X_train.shape}")
print(f"X_test has shape: {X_test.shape}")
print(f"y_train has shape: {y_train.shape}")
print(f"y_test has shape: {y_test.shape}")

y_train = to_categorical(y_train, 43)
y_test = to_categorical(y_test, 43)

data has shape: (5488, 50, 50, 3)
labels have shape: (5488,)
X_train has shape: (4390, 50, 50, 3)
X_test has shape: (1098, 50, 50, 3)
y_train has shape: (4390,)
y_test has shape: (1098,)
(4390, 43)


In [53]:
# Building the model
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=64, kernel_size=(5,5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.15))
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.20))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(rate=0.25))
model.add(Dense(43, activation='softmax'))

# Compilation of the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#M odel display
model.summary() 

C:\Users\rayzh\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 46, 46, 32)     │         2,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 42, 42, 64)     │        51,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 21, 21, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 21, 21, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 19, 19, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 17, 17, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 16384)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     8,389,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 43)             │        22,059 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,833,899 (33.70 MB)

 Trainable params: 8,833,899 (33.70 MB)

 Non-trainable params: 0 (0.00 B)

In [54]:
# Training the Model
with tf.device('/GPU:0'):
    epochs = 35
    history1 = model.fit(X_train, y_train, batch_size=128, epochs=epochs, validation_data=(X_test, y_test))

Epoch 1/35
35/35 ━━━━━━━━━━━━━━━━━━━━ 8s 215ms/step - accuracy: 0.0855 - loss: 14.9181 - val_accuracy: 0.4299 - val_loss: 2.1528
Epoch 2/35
35/35 ━━━━━━━━━━━━━━━━━━━━ 7s 205ms/step - accuracy: 0.5303 - loss: 1.8151 - val_accuracy: 0.8215 - val_loss: 0.8276
Epoch 3/35
35/35 ━━━━━━━━━━━━━━━━━━━━ 7s 204ms/step - accuracy: 0.8068 - loss: 0.6909 - val_accuracy: 0.9308 - val_loss: 0.3511
Epoch 4/35
35/35 ━━━━━━━━━━━━━━━━━━━━ 7s 208ms/step - accuracy: 0.9052 - loss: 0.3525 - val_accuracy: 0.9353 - val_loss: 0.2423
Epoch 5/35
35/35 ━━━━━━━━━━━━━━━━━━━━ 7s 207ms/step - accuracy: 0.9348 - loss: 0.2465 - val_accuracy: 0.9472 - val_loss: 0.2105
Epoch 6/35
35/35 ━━━━━━━━━━━━━━━━━━━━ 7s 211ms/step - accuracy: 0.9421 - loss: 0.2191 - val_accuracy: 0.9599 - val_loss: 0.1975
Epoch 7/35
35/35 ━━━━━━━━━━━━━━━━━━━━ 7s 209ms/step - accuracy: 0.9605 - loss: 0.1597 - val_accuracy: 0.9545 - val_loss: 0.1896
Epoch 8/35
35/35 ━━━━━━━━━━━━━━━━━━━━ 8s 219ms/step - accuracy: 0.9643 - loss: 0.1286 - val_accuracy: 0

In [ ]:
test_data = []
test_id = []

for i in range(len(test_paths)):
    try:
        # open image
        image = Image.open(f"Data/test/{test_paths.iloc[i]}")
        # resize image to (50, 50)
        image = image.resize(rescale_size)
        image = np.array(image)
        test_data.append(image)
        test_id.append(test_index.iloc[i])
    except:
        print(f"error loading image {test_paths.iloc[i]}")

test_data = np.array(test_data)

print(test_data.shape)
print(test_id)


(2353, 50, 50, 3)
[np.int64(661), np.int64(4477), np.int64(1046), np.int64(631), np.int64(6533), np.int64(2899), np.int64(1941), np.int64(5749), np.int64(588), np.int64(1333), np.int64(5826), np.int64(7493), np.int64(4306), np.int64(3429), np.int64(1191), np.int64(4076), np.int64(1601), np.int64(2183), np.int64(4214), np.int64(5366), np.int64(5434), np.int64(6709), np.int64(2421), np.int64(4247), np.int64(5722), np.int64(1115), np.int64(1730), np.int64(6673), np.int64(156), np.int64(2011), np.int64(4416), np.int64(4661), np.int64(6511), np.int64(4411), np.int64(821), np.int64(6246), np.int64(3934), np.int64(5711), np.int64(3233), np.int64(3738), np.int64(4561), np.int64(3780), np.int64(6288), np.int64(3115), np.int64(7155), np.int64(7603), np.int64(1932), np.int64(6387), np.int64(6001), np.int64(4376), np.int64(2393), np.int64(5025), np.int64(660), np.int64(7634), np.int64(1978), np.int64(2398), np.int64(2826), np.int64(2679), np.int64(1466), np.int64(1953), np.int64(3887), np.int64(56

In [ ]:
# predict classes of test data
with tf.device('/GPU:0'):
    pred = np.argmax(model.predict(test_data), axis=-1)

74/74 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step


In [82]:
result = pd.DataFrame({'id': test_index, 'ClassId': pred})
result.to_csv("results.csv", index=False)

# Feature Extraction

### LBP

In [12]:
# hyperparameters
r = 1
points = r * 8
method = 'uniform'

def extract_lbp(source: str):
    # extract image paths

    if source == "train":
        all_images = train_metadata["image_path"]
    else:
        all_images = test_metadata["image_path"]
    
    lbp_data = []
    lbp_images = []

    # extract lbp values
    for path in all_images:
        image = Image.open(f"Data/{source}/{path}")
        image = image.resize(rescale_size)
        gray_image = color.rgb2gray(image)

        lbp = local_binary_pattern(gray_image, points, r, method)

        n_bins = int(lbp.max() + 1)
        hist, _ = np.histogram(lbp.ravel(), bins=n_bins, range=(0, n_bins), density=True)

        lbp_data.append(hist)
        lbp_images.append(path)


    lbp = pd.DataFrame(lbp_data)
    lbp.insert(0, 'image_path', lbp_images)
    
    if source == "train":
        lbp.index = train_metadata.index
    else:
        lbp.index = test_metadata.index

    lbp.to_csv('lbp.csv', index=False)

    return lbp

lbp_train = extract_lbp("train")

c:\Users\rayzh\AppData\Local\Programs\Python\Python312\Lib\site-packages\skimage\feature\texture.py:385: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


### Canny Edge Detection

In [17]:
# Hyperparameter
SIGMA = 1

def extract_canny(source: str):
    if source == "train":
        all_images = train_metadata["image_path"]
    else:
        all_images = test_metadata["image_path"]

    canny_data = []
    canny_images = []

    for path in all_images:
        image = Image.open(f"Data/{source}/{path}")
        image = image.resize(rescale_size)
        image_array = np.array(image)
        gray_image = color.rgb2gray(image_array)

        edges = canny(gray_image, sigma=SIGMA)

        canny_data.append(edges.astype(np.float32)) 
        canny_images.append(path)

    canny_flat = [edge.flatten() for edge in canny_data]
    canny_flat = pd.DataFrame(canny_flat)
    canny_flat.insert(0, 'image_path', canny_images)
    
    if source == "train":
        canny_flat.index = train_metadata.index
    else:
        canny_flat.index = test_metadata.index

    if False:
        canny_entropy(canny_data)

    # for i in range(3):
    #     plt.imshow(canny_data[i], cmap='gray')
    #     plt.title(f"Canny output {i}")
    #     plt.axis('off')
    #     plt.show()    

    return canny_flat

canny_train = extract_canny("train")

In [5]:
# tuning the value of sigma using entropy

def canny_entropy(canny_data):
    entropies = []

    for data in canny_data:
        values, counts = np.unique(data.astype(int), return_counts=True)
        probs = counts / counts.sum() 
        entropies.append(float(entropy(probs, base=2)))

    print(np.mean(entropies)) 
    # sigma = 1: 0.5726
    # sigma = 2: 0.3871
    # sigma = 3: 0.2597

    # choose sigma = 1

In [ ]:
X = canny_train.drop(columns=['image_path']).values  

pca = PCA(n_components=0.75)  # reduce dimensions but keep 75% of the variance
X_pca = pca.fit_transform(X)

print(f"Original shape: {X.shape}")
print(f"Reduced shape: {X_pca.shape}")

canny_df = pd.DataFrame(X_pca, index=canny_train.index)

# Add the image_path column back
canny_df.insert(0, 'image_path', canny_train['image_path'])

canny_df.to_csv('canny.csv', index=False)



Original shape: (2353, 2500)
Reduced shape: (2353, 458)


### Sobel Filtering

In [14]:
def extract_sobel(source: str):

    if source == "train":
        all_images = train_metadata["image_path"]
    else:
        all_images = test_metadata["image_path"]
    
    sobel_data = []
    sobel_images = []

    for path in all_images:
        image = Image.open(f"Data/{source}/{path}")
        image = image.resize(rescale_size)
        image_array = np.array(image)
        gray_image = color.rgb2gray(image_array)

        sobel_edge = sobel(gray_image)

        # sx = sobel_h(gray_image)
        # sy = sobel_v(gray_image)
        # sobel_edge = np.hypot(sx, sy)

        sobel_data.append(sobel_edge.flatten())
        sobel_images.append(path)

    sobel_df = pd.DataFrame(sobel_data)
    sobel_df.insert(0, 'image_path', sobel_images)
    
    if source == "train":
        sobel_df.index = train_metadata.index
    else:
        sobel_df.index = test_metadata.index

    # for i in range(3):
    #     sobel_image = sobel_df.iloc[i].values.reshape(rescale_size)
    #     plt.imshow(sobel_image, cmap='gray')
    #     plt.title(f"Sobel output {i}")
    #     plt.axis('off')
    #     plt.show()

    return sobel_df

sobel_train = extract_sobel("train")

In [ ]:
X = sobel_train.drop(columns=["image_path"]).values  

pca = PCA(n_components=0.75)  
X_pca = pca.fit_transform(X)

print(f"Original shape: {X.shape}")
print(f"Reduced shape: {X_pca.shape}")

sobel_df = pd.DataFrame(X_pca, index=sobel_train.index)
sobel_df.insert(0, 'image_path', sobel_train["image_path"])

sobel_df.to_csv('sobel.csv', index=False)

Original shape: (2353, 2500)
Reduced shape: (2353, 101)


## Scaling Data

In [6]:
train_add_features = pd.read_csv("Data/train/Features/additional_features.csv").drop(columns=['image_path'])
train_edge_density = train_add_features[["edge_density"]]
train_mean_rgb = train_add_features.drop(columns=["edge_density"])
train_canny = pd.read_csv("Data/train/Features/canny.csv").drop(columns=['image_path'])
train_color_hist = pd.read_csv("Data/train/Features/color_histogram.csv").drop(columns=['image_path'])
train_hog_pca = pd.read_csv("Data/train/Features/hog_pca.csv").drop(columns=['image_path'])
train_lbp = pd.read_csv("Data/train/Features/lbp.csv").drop(columns=['image_path'])
train_sobel = pd.read_csv("Data/train/Features/sobel.csv").drop(columns=['image_path'])


scaler1 = StandardScaler()
train_edge_density_scaled = scaler1.fit_transform(train_edge_density)

scaler2 = StandardScaler()
train_mean_rgb_scaled = scaler2.fit_transform(train_mean_rgb)

scaler3 = StandardScaler()
train_canny_scaled = scaler3.fit_transform(train_canny)

scaler4 = StandardScaler()
train_color_hist_scaled = scaler4.fit_transform(train_color_hist)

scaler5 = StandardScaler()
train_hog_pca_scaled = scaler5.fit_transform(train_hog_pca)

scaler6 = StandardScaler()
train_lbp_scaled = scaler6.fit_transform(train_lbp)

scaler7 = StandardScaler()
train_sobel_scaled = scaler7.fit_transform(train_sobel)


## Training SVM on canny and color hist

In [74]:
canny_train = extract_canny("train")
train_color_hist = pd.read_csv("Data/train/Features/color_histogram.csv")
train_color_hist.index = train_metadata.index

In [83]:
DEGREE = 1000
pca = PCA(n_components = DEGREE)


def apply_pca(data):

    X = data.drop(columns=["image_path"])

    data_pca = pca.fit_transform(X)
    data_pca = pd.DataFrame(data_pca, index=data.index)
    data_pca.insert(0, 'image_path', data['image_path'])

    return data_pca

canny_train_pca = apply_pca(canny_train)


In [84]:
train_canny_color = pd.concat([canny_train_pca, train_color_hist], axis=1)
train_canny_color.columns = train_canny_color.columns.astype(str)
train_canny_color = train_canny_color.drop(columns=["image_path"])
train_canny_color

,0,1,2,3,4,5,6,7,8,9,...,ch_86,ch_87,ch_88,ch_89,ch_90,ch_91,ch_92,ch_93,ch_94,ch_95
id,,,,,,,,,,,,,,,,,,,,,
7218,-2.051437,-0.669757,-0.315596,0.801923,-0.853103,0.463098,-0.606504,0.774105,0.931920,-0.480700,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6333,-2.525180,0.028389,-0.416241,-1.058156,-0.676034,0.897606,1.331595,1.021448,0.557510,1.062243,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6867,2.291903,-0.143319,2.981271,-0.610320,1.993576,4.577984,-2.032192,2.292870,0.467942,0.524660,...,0.062342,0.056108,0.049874,0.043639,0.061303,0.072732,0.082084,0.110137,0.156894,0.670176
7730,-1.399055,4.003945,-0.877522,-1.250140,1.936476,0.971122,-0.085330,-0.685970,-0.353868,0.193804,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6338,0.134350,1.608424,1.234319,-1.729691,0.694583,-3.498270,1.538762,2.223016,0.389273,0.170613,...,0.013489,0.023231,0.016487,0.008243,0.025479,0.023981,0.011241,0.006745,0.006745,0.014988
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,-0.835904,-0.338931,0.450743,-0.228883,-0.453278,0.557945,2.058186,0.317780,0.399139,-1.141895,...,0.010370,0.017284,0.019358,0.025580,0.021432,0.047013,0.070519,0.105087,0.152100,0.763265
4084,0.767188,1.727126,3.389711,1.114812,2.352844,1.234217,0.578990,-1.399358,-0.018394,-1.250483,...,0.240318,0.059315,0.034855,0.012230,0.008561,0.011007,0.009784,0.010395,0.009784,0.201182
5117,7.668096,-2.868917,5.046779,2.300111,3.725677,-2.759581,4.649038,-1.302922,-0.842877,3.635046,...,0.019605,0.020726,0.030248,0.021286,0.029128,0.024086,0.047613,0.098026,0.185409,0.949452


In [86]:
SVM_model = SVC(kernel="linear", C=1.0)
rfe = RFE(estimator=SVM_model, n_features_to_select=300, step=50)

rfe.fit(train_canny_color, train_metadata["ClassId"])

col = rfe.get_support(indices=True)

In [87]:
train_data = train_canny_color.iloc[:, col]
train_data

,0,1,2,3,5,6,7,8,9,10,...,ch_37,ch_38,ch_63,ch_64,ch_65,ch_66,ch_67,ch_68,ch_69,ch_70
id,,,,,,,,,,,,,,,,,,,,,
7218,-2.051437,-0.669757,-0.315596,0.801923,0.463098,-0.606504,0.774105,0.931920,-0.480700,2.462802,...,0.256266,0.538795,0.000000,0.000000,0.004936,0.315108,0.278908,0.401495,0.321690,0.440164
6333,-2.525180,0.028389,-0.416241,-1.058156,0.897606,1.331595,1.021448,0.557510,1.062243,0.484856,...,0.398772,0.132388,0.000000,0.000000,0.000000,0.081961,0.297379,0.517149,0.529480,0.393846
6867,2.291903,-0.143319,2.981271,-0.610320,4.577984,-2.032192,2.292870,0.467942,0.524660,-0.777841,...,0.217631,0.269911,0.233436,0.001039,0.269109,0.215080,0.098708,0.087279,0.121567,0.217158
7730,-1.399055,4.003945,-0.877522,-1.250140,0.971122,-0.085330,-0.685970,-0.353868,0.193804,-0.917611,...,0.084188,0.047046,0.000000,0.000000,0.474292,0.781341,0.108681,0.192038,0.305995,0.073333
6338,0.134350,1.608424,1.234319,-1.729691,-3.498270,1.538762,2.223016,0.389273,0.170613,5.642231,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.007494,0.211330,0.172361,0.194843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,-0.835904,-0.338931,0.450743,-0.228883,0.557945,2.058186,0.317780,0.399139,-1.141895,2.118095,...,0.441300,0.494900,0.378768,0.000000,0.000000,0.016593,0.178372,0.101630,0.273780,0.250273
4084,0.767188,1.727126,3.389711,1.114812,1.234217,0.578990,-1.399358,-0.018394,-1.250483,1.175099,...,0.000000,0.000000,0.181204,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5117,7.668096,-2.868917,5.046779,2.300111,-2.759581,4.649038,-1.302922,-0.842877,3.635046,-2.213576,...,0.136507,0.110009,0.865617,0.000560,0.002241,0.044812,0.100827,0.081222,0.091864,0.069458


In [88]:
SVM = SVC(kernel="linear", C=1.0)
SVM.fit(train_data, train_metadata["ClassId"])

SVC(kernel='linear')

In [89]:
canny_test = extract_canny("test")
test_color_hist = pd.read_csv("Data/test/Features/color_histogram.csv")
test_color_hist.index = test_metadata.index

In [90]:
canny_test_pca = apply_pca(canny_test)
test_canny_color = pd.concat([canny_test_pca, test_color_hist], axis=1)
test_canny_color.columns = test_canny_color.columns.astype(str)
test_canny_color = test_canny_color.drop(columns=["image_path"])
test_canny_color

,0,1,2,3,4,5,6,7,8,9,...,ch_86,ch_87,ch_88,ch_89,ch_90,ch_91,ch_92,ch_93,ch_94,ch_95
0,3.972194,0.372827,-1.312706,0.921197,-0.936266,-1.190727,1.052612,1.272004,-1.311363,3.494884,...,0.059833,0.076151,0.062164,0.090137,0.125882,0.093246,0.087806,0.076928,0.096354,0.840764
1,4.428937,-2.584756,1.242029,2.571882,0.195155,0.276150,-0.090557,-0.779253,2.988510,0.004688,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,3.410320,-3.148561,-0.090595,-2.218615,-1.177240,-0.457714,-1.201756,1.831517,-0.610035,0.224229,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,-3.074435,1.948776,-0.925813,1.062414,-1.761376,-2.204464,-0.819742,-0.378842,-1.077743,-2.029878,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,-0.731861,4.398859,0.981835,-1.039885,-0.704322,1.110292,2.530677,0.198082,-0.466632,-1.416666,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2348,-0.224179,1.002199,-0.207040,0.937442,-1.572850,0.669617,-0.862222,-0.959062,0.469508,-1.186071,...,0.005119,0.001463,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2349,-1.112046,1.533942,-1.101301,-3.926783,3.161074,2.479483,-1.725642,-0.946823,0.780640,-2.549285,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2350,1.800065,0.597188,-2.726380,1.045198,-2.418544,0.566243,-1.156263,-1.097498,0.850812,1.286888,...,0.066123,0.059110,0.044082,0.040074,0.096179,0.100186,0.042078,0.052097,0.029054,0.043080
2351,2.402601,-3.318600,0.953507,0.732682,-1.205913,4.622538,-0.337615,-1.107822,-0.101137,-0.329561,...,0.040461,0.093661,0.095909,0.039712,0.016484,0.009741,0.004496,0.008242,0.025476,0.182076


In [91]:
test_data = test_canny_color.iloc[:, col]
predictions = SVM.predict(test_data)
predictions

array([28, 16, 21, ...,  3,  4, 11])

In [93]:
result = pd.DataFrame({'id': test_index, 'ClassId': predictions})
result.to_csv("results.csv", index=False)